In [19]:
'''
    Read the dataset from tsv file.
'''
import csv

data = {}

with open("data/tgif-v1.0.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    count = 0
    for idx, row in enumerate(rd):
      if count < 100000:
        data[idx] = ({ 'url': row[0], 'sentence': row[1]})
        count+= 1
      else:
        break



In [20]:
data[24999]
len(data)

100000

In [21]:
'''
    Removes stop words from sentences
'''

# install nltk if not present
try:
    import nltk
    import ssl

    nltk.download('stopwords')
except ImportError as e:
    !pip3 install nltk
    import nltk
    nltk.download('stopwords')

from nltk.corpus import stopwords

print(set(stopwords.words('english')))
english_stop_words = stopwords.words('english')

for key, value in data.items():
    value['sentence'] = ' '.join(word for word in value['sentence'].split() if word not in english_stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yashkothari/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{"you'd", 'his', 'on', 'me', 'so', 'its', 'mightn', 'her', 't', "you'll", "haven't", 'few', 'our', 'ma', 'further', 'they', 'between', 'wasn', 're', 'm', 'while', 'just', 'about', 'yours', 'will', 'below', 'isn', 've', 'very', "needn't", 'couldn', 'won', 'this', 'such', "shouldn't", 'there', "should've", 'shouldn', 'yourself', 'because', 'being', 'only', 'shan', 'nor', 'when', 'most', 'under', 'd', "didn't", "doesn't", 'have', 'a', 'at', "she's", 'were', 'with', 'until', 'them', 'been', 'off', "hasn't", 'we', 'how', 'don', 'down', 'not', 'do', 'than', 'you', 'doesn', 'himself', "isn't", 'mustn', 'the', "couldn't", 'after', 'she', 'each', 'own', 'where', 'some', 'aren', 'my', "shan't", 'by', 'to', 'or', 'of', 'haven', 'ourselves', 'myself', 'am', "weren't", "won't", 'are', 'didn', 'it', 'and', 'hasn', "mightn't", 'which', 'up', 'weren', 'does', 'if', "wouldn't", 'again', 'itself', "it's", 'through', 'in', 'any', 'whom', 'having', 'yourselves', 'both', 'more', 'those', 'but', 'themselves

In [22]:
'''
    Removes punctuation from sentences
'''
import re

for key, value in data.items():
    value['sentence'] = re.sub(r'[^\w\s]', '', value['sentence'])


In [23]:
'''
  Create reverse index for each word to its document id for easier retrieval
'''
# from nltk.tokenize import word_tokenize
# nltk.download('punkt')

# index = {}
# for key, value in data.items():
#   tokens = word_tokenize(value['sentence'])
#   for token in tokens:
#     doc_ids = index.get(token, set())
#     doc_ids.add(key)
#     index[token] = doc_ids

# print("Total keys in index: " + str(len(index.keys())))
# print("Top 50: ")
# # print(list(index.items())[:50])

'\n  Create reverse index for each word to its document id for easier retrieval\n'

In [24]:
for i in range(len(data)):
    data[i]['id'] = str(i)

In [25]:
import math

indexTable = dict()
allGIFs = dict()

for gif in data.values():
    tokens = gif['sentence'].split()
    allGIFs[gif['id']] = dict()
    for term in tokens:
        if term in allGIFs[gif['id']]:
            allGIFs[gif['id']][term] += 1
        else:
            allGIFs[gif['id']][term] = 1

    uniqueTokens = list(dict.fromkeys(tokens))
    for word in uniqueTokens:
        if word in indexTable:
            indexTable[word] += 1
        else:
            indexTable[word] = 1

tfIdf = dict()
for gif in data.values():
    tfIdf[gif['id']] = dict()
    for word in allGIFs[gif['id']]:
        tfIdf[gif['id']][word] = allGIFs[gif['id']][word] * \
            math.log10(allGIFs.__len__()/indexTable[word])

for item in tfIdf:
    print("In file " + str(item) + "---> " + str(tfIdf[item]))



In [ ]:
finalTable = dict()

j = 0
for index in allGIFs:
    finalTable[index] = dict()
    for word in indexTable:
        if word in allGIFs[index]:
            finalTable[index][word] = tfIdf[index][word]
        else:
            finalTable[index][word] = 0
        
    print("Iteration " + str(j))
    print("---------------")
    j += 1

Iteration 0
---------------
Iteration 1
---------------
Iteration 2
---------------
Iteration 3
---------------
Iteration 4
---------------
Iteration 5
---------------
Iteration 6
---------------
Iteration 7
---------------
Iteration 8
---------------
Iteration 9
---------------
Iteration 10
---------------
Iteration 11
---------------
Iteration 12
---------------
Iteration 13
---------------
Iteration 14
---------------
Iteration 15
---------------
Iteration 16
---------------
Iteration 17
---------------
Iteration 18
---------------
Iteration 19
---------------
Iteration 20
---------------
Iteration 21
---------------
Iteration 22
---------------
Iteration 23
---------------
Iteration 24
---------------
Iteration 25
---------------
Iteration 26
---------------
Iteration 27
---------------
Iteration 28
---------------
Iteration 29
---------------
Iteration 30
---------------
Iteration 31
---------------
Iteration 32
---------------
Iteration 33
---------------
Iteration 34
-----------

In [ ]:
query = input("enter your query").lower()
queryTokens = query.split()

queryVector = {i: queryTokens.count(i) for i in queryTokens}

for item in queryVector:
    if item in indexTable:
        queryVector[item] = queryVector[item] * \
            math.log10((allGIFs.__len__() + 1) / (indexTable[item] + 1))
    else:
        queryVector[item] = queryVector[item] * \
            math.log10(allGIFs.__len__() + 1)

print(queryVector)

{'dog': 1.5370207594037937}


In [ ]:
finalQuery = dict()

for word in indexTable:
    if word in queryVector:
        finalQuery[word] = queryVector[word]
    else:
        finalQuery[word] = 0

In [ ]:
distance_dict = {}

query_denom = 0
for word, value in finalQuery.items():
  query_denom += value**2 

for idx, doc_vector in finalTable.items():
  doc_denom = 0
  for word, value in doc_vector.items():
    doc_denom += value**2 
  
  denom = math.sqrt(query_denom * doc_denom)

  numerator=0
  for word, value in finalQuery.items():
    numerator += (doc_vector.get(word, 0) * value)
  
  distance_dict[idx] = numerator / denom # cosine similarity


In [ ]:
import operator 
ranklist = sorted(distance_dict.items(), key=operator.itemgetter(1), reverse=True)

print(ranklist[:25])

[('15500', 0.7305515532158706), ('7197', 0.6923265664939596), ('4386', 0.6321187294039263), ('16362', 0.6280515095018487), ('19532', 0.607910301163241), ('8461', 0.6020194695012334), ('8760', 0.5900174527484013), ('24626', 0.5864410905777467), ('16108', 0.5855470994974861), ('2770', 0.584533689781379), ('8127', 0.5759077889972329), ('6297', 0.5671504473092213), ('18398', 0.5665858813354183), ('17951', 0.5663487343751334), ('10667', 0.5638032969974648), ('3202', 0.5568156673489203), ('4868', 0.5560407771766361), ('10851', 0.5441632011385512), ('23576', 0.542658551503791), ('20923', 0.5238581662372729), ('20744', 0.5238477124615806), ('12827', 0.5189112895293161), ('2751', 0.5179246233485207), ('15499', 0.5171841794029454), ('15889', 0.5167102810222013)]


In [ ]:
# '''
#   Code fragment for displaying GIF
# '''
# from IPython.display import IFrame

# # GIF link used as IFrame (need to change path as needed)
# index = 0
# iframe_url = data[int(ranklist[index][0])]['url']

# # resized output IFrame
# iframe= IFrame(src=iframe_url, width=600, height=550)

In [ ]:
from IPython.display import HTML

scafolding = '<iframe src={url} width=700 height=350></iframe>'
iframe_string = ''

for idx,item in enumerate(ranklist[0:10]):
  iframe_url = data[int(ranklist[idx][0])]['url']
  iframe_string += scafolding.format(url=iframe_url)
HTML(iframe_string)

/Users/yashkothari/Library/Python/3.8/lib/python/site-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
